In [1]:
!pip install datasets evaluate
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

# Import necessary libraries


In [2]:
import json
import pandas as pd
import numpy as np
from google.colab import drive
import requests
import zipfile
import os
from transformers import RobertaForSequenceClassification, RobertaTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset
import random
import torch
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score
import evaluate

# downloading dataset

In [ ]:
drive.mount('/content/drive')

In [ ]:
url = 'https://cims.nyu.edu/~sbowman/multinli/multinli_1.0.zip'
response = requests.get(url)

output_path = '/content/drive/My Drive/dataset.zip'
with open(output_path, 'wb') as f:
  f.write(response.content)

In [ ]:
dataset_path = '/content/drive/My Drive/dataset.zip'
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
  zip_ref.extractall('/content/drive/My Drive/multinli_1.0')

In [ ]:
extracted_path = '/content/drive/My Drive/multinli_1.0/multinli_1.0'
print(os.listdir(extracted_path))

In [ ]:
train_file_path = '/content/drive/MyDrive/multinli_1.0/multinli_1.0/multinli_1.0_train.jsonl'
train_df = pd.read_json(train_file_path, lines=True)

In [ ]:
train_df.head()

In [ ]:
matched_val_path = '/content/drive/MyDrive/multinli_1.0/multinli_1.0/multinli_1.0_dev_matched.jsonl'
val_match_df = pd.read_json(matched_val_path, lines=True)
val_match_df.shape

In [ ]:
mismatched_val_path = '/content/drive/MyDrive/multinli_1.0/multinli_1.0/multinli_1.0_dev_mismatched.jsonl'
val_mismatched_df = pd.read_json(mismatched_val_path, lines=True)
val_mismatched_df.shape

In [ ]:
for col in train_df.columns.to_list():
  print(f'{col}\n {train_df.iloc[10][col]}')

# Roberta large model

In [3]:
model_name_or_path = "roberta-large"
num_epochs = 1
lr = 1e-5
batch_size = 4

In [4]:
dataset = load_dataset('multi_nli')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [5]:
metric = evaluate.load('accuracy')

In [6]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
def sample_10_percent(dataset):
    total_size = len(dataset)
    sample_size = max(1, total_size // 10)
    indices = random.sample(range(total_size), sample_size)
    return dataset.select(indices)

In [8]:
sampled_train_dataset = sample_10_percent(dataset['train'])
sampled_validation_dataset = sample_10_percent(dataset['validation_matched'])
sampled_train_dataset.shape, sampled_validation_dataset.shape

((39270, 10), (981, 10))

In [9]:
tokenizer = RobertaTokenizer.from_pretrained(model_name_or_path, padding_side="right", num_labels=3)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [10]:
def preprocess_function(examples):
    outputs = tokenizer(examples['premise'], examples['hypothesis'], truncation=True, max_length=None)
    return outputs

In [11]:
train_tokenized_dataset = sampled_train_dataset.map(preprocess_function, batched=True, remove_columns=["promptID", "pairID", "premise", "premise_binary_parse", "premise_parse", "hypothesis", "hypothesis_binary_parse", "hypothesis_parse", "genre"])
train_tokenized_dataset = train_tokenized_dataset.rename_column("label", "labels")

Map:   0%|          | 0/39270 [00:00<?, ? examples/s]

In [12]:
match_val_tokenized_dataset = sampled_validation_dataset.map(preprocess_function, batched=True, remove_columns=["promptID", "premise", "pairID", "premise_binary_parse", "premise_parse", "hypothesis", "hypothesis_binary_parse", "hypothesis_parse", "genre"])
match_val_tokenized_dataset = match_val_tokenized_dataset.rename_column("label", "labels")

Map:   0%|          | 0/981 [00:00<?, ? examples/s]

In [13]:
train_tokenized_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 39270
})

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [15]:
model = RobertaForSequenceClassification.from_pretrained(model_name_or_path, num_labels=3).to('cuda')

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="pouya/roberta-large-peft-lora",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=batch_size,
    num_train_epochs=num_epochs,
    warmup_steps=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
    report_to="none",
    logging_steps= 100,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=match_val_tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.378700,0.318748,0.881753


TrainOutput(global_step=2454, training_loss=0.510224560750823, metrics={'train_runtime': 2213.485, 'train_samples_per_second': 17.741, 'train_steps_per_second': 1.109, 'total_flos': 4321984634022744.0, 'train_loss': 0.510224560750823, 'epoch': 0.9997962925239356})

In [22]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [23]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
num_trainable_params = count_trainable_parameters(model)
print(f"Number of trainable parameters: {num_trainable_params}")

Number of trainable parameters: 355362819
